<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# *Linja*
### *Sistemas Inteligentes* (Curso 2023-2024)



<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Hacia la versión "automática" del juego (II) </h2>

## Docentes

 - Pedro Latorre Carmona
 
## Alumno

 - Christian Núñez Duque

## Definición de la clase

Partimos de la definición de la clase **Linja** que se ha estado viendo en las semanas anteriores, y tenemos que introducir el método de **MINIMAX**.

## Definición de la clase Linja para jugar manualmente

In [1]:
# 
# Autor: Christian Núñez Duque
# Fecha: 08/12/2023
# Versión: 1.0
# Con Run All se puede jugar. El humano es el jugador 1 (Negras) y el ordenador el 2 (Rojas).
#

from copy import deepcopy
from typing import Tuple, List
from IPython.display import display
from ipywidgets import HTML
import copy
import math

class Linja:
    
    # Inicializar los atributos de la clase
    def __init__(self, matrix):
        # Matriz del objeto
        self.setMatrix(matrix)
        # Variable que contiene el número del movimiento del turno actual (0: primer movimiento; 1: segundo movimiento)
        self.movement = 0
        # Variable que contiene el número del jugador actual (1: Negro; 2: Rojo)
        self.turnoActual = 1
        # Variable que contiene el valor del desplazamiento entre filas para el primer movimiento (1 o -1)
        self.mov1row = 0
        # Variable que almacena las fichas que hay en una fila al hacer el primer movimiento. Se usa para el segundo.
        self.numFilas1 = 1
        # Variable que determina si el movimiento extra está disponible.
        self.movExtra = True
        # Variable que contiene la suma de costes con valor 5 del jugador rojo
        self.costeRojo = 0
        # Variable que contiene la suma de costes con valor 5 del jugador negro
        self.costeNegro = 0
      
    # Método utilizado para asignar la matriz a una instancia al inicialiarla.
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)
    
    # Método que devuelve la matriz de la instancia.
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)
    
    # Método que coloca una ficha en una casilla.
    def placeTile(self, row: int, col: int, tile: int):
        self.matrix[row][col] = tile
    
    # Método que elimina una ficha de una casilla (vacía la casilla)
    def deleteTile(self, row: int, col: int):
        self.matrix[row][col] = 0
    
    # Método que calcula el coste del jugador pasado por parámetro
    def cost(self, playerID: int) -> int:
        coste = 0
        inicio = 0
        limite = 0
        valores_filas = {
            0: 5,
            1: 3,
            2: 2,
            3: 1,
            4: 1,
            5: 2,
            6: 3,
            7: 5
        }
        if(playerID==1):
            inicio = 4
            limite = 8
        else:
            inicio = 0
            limite = 4
        for i in range (inicio,limite):
            for j in range (0,6):
                if(self.matrix[i][j]==playerID):
                    coste+=valores_filas[i]
        return coste
                
    # Método que calcula la función de utilidad del estado actual. Se maximiza para el jugador rojo (ordenador).
    def utility(self) -> int:
        resultado = 0
        resultado = (self.cost(2)+self.costeRojo) - (self.cost(1)+self.costeNegro)
        return resultado

    def get_content(self, row: int, col: int, validate: bool):
        contenido = [None]
        if self.matrix[row][col] == 0:
            contenido[0] = "casillavacia"
        elif self.matrix[row][col] == 1:
            contenido[0] = "casillanegra"
        elif self.matrix[row][col] == 2:
            contenido[0] = "casillaroja"
            
        if validate:
            available = self.availableMovs(1)

            found = False
            for tupla in available:
                r, c = tupla  
                if r == row and c == col:
                    contenido[0] = "casilladisp"
                    found = True
                    break

            if found:
                self.matrix[row][col] = "casilladisp"
            
        return contenido
    
    def get_html(self, validate: bool):
        element_image = {
            "casillavacia": "./ImagenesCasillasLinja/CasillaVacia.png",
            "casillanegra": "./ImagenesCasillasLinja/CasillaNegra.png",
            "casillaroja": "./ImagenesCasillasLinja/CasillaRoja.png",
            "casilladisp": "./ImagenesCasillasLinja/CasillaDisp.png"
        }

        html_string = "<style> img.game {width: 47px !important; height: 37px !important;}</style><table>"

        new_row = "<tr>"
        end_row = "</tr>"

        for i in range(8):
            html_string+=new_row
            for j in range(6):

                content = self.get_content(i,j,validate)
                drawing = element_image[content[0]]

                html = '<td><img class="game" src=%s alt=""></img></td>' % drawing     


                html_string+=html
            html_string+=end_row

        html_string += "</table>"


        return html_string

    
    # Método que valida los movimientos. Para ello comprueba si la celda llamada contiene una ficha del jugador
    # actual y llama a mov1 si es el movimiento 1 o a mov2 si es el movimiento 2.
    #
    # Acceso es una variable que si es True y el movimiento es legal, hace el movimiento. Si es False, solo muestra
    # si el movimiento sería válido.
    def validateMov(self, row: int, col: int, acceso: bool) -> bool:
        #Comprobación de que existe la ficha a mover.
        if(self.matrix[row][col] == self.turnoActual):   
            #Caso 1: Si es el primer movimiento.
            if(self.movement==0):
                return self.mov1(row, col, acceso)
            #Caso 2: Si es el segundo movimiento.
            elif(self.movement==1):
                return self.mov2(row, col, acceso)
        return False

    
    # Método que se encarga de validar (y realizar) el movimiento 1. Para ello se comprueban las reglas de Linja.
    def mov1(self, row: int, col: int, acceso: bool) -> bool:
        #Caso 1.1: Si la fila siguiente está fuera del tablero.
        if(row+self.numFilas1 < 0 or row+self.numFilas1 > 7):
            return False
        #Excepción 1.1: Si la celda no está ocupada y es una fila final (Jugador 1).
        elif(row+self.numFilas1 == 7 and self.matrix[row+self.numFilas1][col]==0):
            if(acceso):    
                self.costeNegro+=5
                self.mov1row = 1
                self.placeTile(row+self.numFilas1, col, self.turnoActual)
            return True
        #Excepción 1.2: Si la celda no está ocupada y es una fila final (Jugador 2).  
        elif(row+self.numFilas1 == 0 and self.matrix[row+self.numFilas1][col]==0):
            if(acceso):
                self.costeRojo+=5
                self.mov1row = 1
                self.placeTile(row+self.numFilas1, col, self.turnoActual)
            return True
        #Caso 1.2: Si la celda no está ocupada y no es una fila final.
        elif(self.matrix[row+self.numFilas1][col]==0):
            if(acceso):
                self.placeTile(row+self.numFilas1, col, self.turnoActual)
                self.mov1row = self.numMovs(row+self.numFilas1)-1
                #Caso 1.3: Si el primer movimiento es a una fila vacía.
                if(self.numMovs(row+self.numFilas1)==1):
                    if(acceso):
                        self.gestorMov()
            return True
        #Excepción 2: Si el primer movimiento es a una fila final ocupada.
        elif(self.matrix[row+self.numFilas1][col]!=0 and (row+self.numFilas1==7 or row+self.numFilas1==0)):
            if(acceso):    
                if(self.turnoActual==1):
                    self.costeNegro+=5
                    self.mov1row = 1
                else:
                    self.costeRojo+=5
                    self.mov1row = 1
            return True
        #Caso 1.3: Si el primer movimiento es a una celda ocupada.
        elif(self.matrix[row+self.numFilas1][col]!=0 and (row+self.numFilas1!=7 or row+self.numFilas1!=0)):
            newCol=self.availableCols(row+self.numFilas1, col)
            #Si no hay casillas disponibles no es un movimiento válido.
            if(newCol==-1):
                    return False
            if(acceso):
                self.placeTile(row+self.numFilas1, newCol, self.turnoActual)
                self.mov1row = self.numMovs(row+self.numFilas1)-1
            return True
        return False    
    
    # Método que se encarga de validar (y realizar) el movimiento 2. Para ello se comprueban las reglas de Linja.
    def mov2(self, row: int, col: int, acceso: bool) -> bool:
        #Caso 2.1: Si la fila siguiente está fuera del tablero.
        if(row+self.mov1row < 0 or row+self.mov1row > 7):
            return False
        #Caso 2.2: Si el movimiento es a una fila vacía.
        elif(self.numMovs(row+self.mov1row) == 0):
            #Movimiento extra: No se cambia el turno si es la primera vez que pasa.
            if(acceso):    
                if(self.movExtra):
                    self.movement = -1
                    self.movExtra = False
                self.placeTile(row+self.mov1row, col, self.turnoActual)
            return True
        #Caso 2.3: Si es a una fila final.
        elif(row+self.mov1row==0 or row+self.mov1row==7):
            if(acceso):    
                if(self.turnoActual==1):
                    self.costeNegro+=5
                else:
                    self.costeRojo+=5    
            return True
        #Caso 2.4: Si es a una casilla cualquiera ocupada.
        elif(self.matrix[row+self.mov1row][col]!=0 and (row+self.mov1row!=0 or row+self.mov1row!=7)):
            newCol=self.availableCols(row+self.mov1row, col)
            #Si no hay casillas disponibles no es un movimiento válido.
            if(newCol==-1):
                return False
            if(acceso):
                self.placeTile(row+self.mov1row, newCol, self.turnoActual)
            return True
        #Caso 2.5: Si es a una casilla cualquiera vacía.
        elif(self.matrix[row+self.mov1row][col]==0):
            if(acceso):    
                self.placeTile(row+self.mov1row, col, self.turnoActual)
            return True
        return False
    
    # Método que devuelve el número de fichas que hay en la fila pasada por parámetro.
    def numMovs(self, row: int) -> int:
        contador=0
        for j in range(0, 6):
            if(self.matrix[row][j] != 0):
                contador+=1
        return contador
    
    # Método que devuelve la columna más cercana vacía de la fila pasada por parámetro. 
    # Si toda la fila está ocupada devuelve -1.
    def availableCols(self, row: int, col: int) -> int:
        freeCol=-1
        inc=1
        if(self.numMovs(row)==6):
            return freeCol
        for i in range (0,6):
            if(self.matrix[row][i]==0):
                return i            
            freeCol = i
        return freeCol
    
    # Método que devuelve una lista de tuplas con los movimientos disponibles.
    def availableMovs(self, playerID: int) -> List[Tuple]:
        available = []
        for i in range (0,8):
            for j in range (0,6):
                if (self.validateMov(i, j, False)):
                    available.append((i, j))
        return available
        
    
    # Método que se encarga de mover una ficha. Para ello, valida el movimiento, y si es válido, mueve la ficha.
    def moverFicha(self, row:int, col: int, acceso: bool) -> bool:
        if(self.validateMov(row, col, acceso)):
            self.deleteTile(row, col)
            self.gestorMov()
            return True
        else:

            if(self.turnoActual == 1):
                available = self.availableMovs(self.turnoActual)
                print("Los movimientos disponibles son:")
                print(available)
            return False
                
    # Método que gestiona los cambios de variables al hacer un movimiento.
    def gestorMov(self):
        if(self.movement==1):
            self.movement=0
            self.cambiarTurno()
        else:
            if(self.turnoActual==2):
                self.mov1row=0-self.mov1row
            self.movement+=1
    
    # Método que gestiona los cambios de turno entre los jugadores.
    def cambiarTurno(self):
        if(self.turnoActual==1):
            self.numFilas1=-1
            self.turnoActual=2
        else:
            self.numFilas1=1
            self.turnoActual=1
        self.movExtra=True
    
    # Método que calcula el ganador en un estado final del juego.
    def endGame(self):
        if (self.utility() > 0):
            victory = 1
        else:
            victory = 2
        print("Final del juego. El ganador es", victory)
    
    # Método que comprueba si el juego ha acabado.
    def gameOver(self) -> bool:
        indexBlack = []
        indexRed = []

        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                if (self.matrix[i][j] == 1):
                    indexBlack.append(i)
                elif (self.matrix[i][j] == 2):
                    indexRed.append(i)

        if len(indexBlack) == 0:
            return True 

        max_indice_1 = min(indexBlack)
        max_indice_2 = max(indexRed)
        
        return max_indice_1 > max_indice_2
    
    # Método que imprime por pantalla la matriz de un estado en modo texto.
    def toString(self):
        for i in range (0,8):
            print(self.matrix[i])
        print("")
        
    # Método que genera los estados hijos de una instancia de la clase para un jugador.    
    def generateSuccessors(self, playerID: int) -> List['Linja']:
        result = []
        for row1 in range(8):
            for col1 in range(6):
                tablero_copia = Linja(self.matrix)  
                if(playerID==2):
                    tablero_copia.cambiarTurno()
                if tablero_copia.validateMov(row1, col1, False):
                    tablero_copia.moverFicha(row1, col1, True)
                    for row2 in range(8):
                        for col2 in range(6):
                            tablero_copia_2 = copy.deepcopy(tablero_copia) 
                            if tablero_copia_2.validateMov(row2, col2, False):
                                tablero_copia_2.moverFicha(row2, col2, True)
                                #Límite de 6 estados hijos
                                if(len(result)==6):
                                    break
                                result.append(tablero_copia_2)
        return result


## Minimax

Algoritmo Minimax con poda alfa-beta que se encarga de devolver la matriz con mayor coste para un jugador, además del coste y de una variable de parada.

In [2]:
# Algoritmo Minimax con poda alfa-beta que se encarga de devolver la matriz con mayor coste para un jugador, además del coste y de una variable de parada.
def miniMax(state:Linja, currentLevel:int, maxLevel:int, playerID:int, alpha:int, beta:int, stop:bool) -> Tuple[Linja, int, bool]:

    possibleMove = True
    
    if(len(state.availableMovs(playerID))==0): 
        possibleMove = False
        
    if (possibleMove == False or currentLevel == maxLevel):
        return (state.matrix,state.utility(),stop)  
    
    successorMatrices = state.generateSuccessors(playerID)
        
    # Si no hay sucesores se para
    if len(successorMatrices) == 0:
        stopDigging = True
        return (state.matrix,state.utility(),stopDigging)
    
    bestMatrix = None
    
    if playerID == 2:                    
        maxValue = -math.inf #alpha
        for i in range(0, len(successorMatrices)):            
                
                mat = successorMatrices[i]
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 1, alpha,beta,stop)
                
                best = utility 
            
                if best > maxValue:
                    maxValue = best
                    bestMatrix = successorMatrices[i].getMatrix()
                    
                alpha = max(alpha, best)                
                if best >= beta:
                    return (matrizS,best,stop)
                    
    else:                           
        minValue = math.inf #beta
        for i in range(0, len(successorMatrices)):
            
                
                mat = successorMatrices[i]
                matrizS, utility, stop = miniMax(mat, currentLevel + 1, maxLevel, 2, alpha,beta,stop)
            
            
                if utility < minValue:
                    minValue = utility
                    bestMatrix = successorMatrices[i].getMatrix()    
                
                beta = min(beta, utility)
                if utility <= alpha:                    
                    return (matrizS,utility,stop)
                    
    return (bestMatrix,utility,stop)


## Poner en funcionamiento Minimax

In [3]:
# Método que ejecuta minimax para obtener el estado siguiente determinado por minimax
def performActionMinMax(state:Linja, player:int):
    
    matrizB=state.getMatrix()
    tmpMatriz = [row[:] for row in matrizB] 
           
    depth = 2
    
    matrizoptima = tmpMatriz
    stop = False 
    currentLevel = 0    
    itera = 0  
        
    while not stop and itera <= 3:          
        tmpMatrizB = Linja(tmpMatriz)
        
        (matrizoptima, valoroptimo, stop) = miniMax(tmpMatrizB, currentLevel, depth, player, -math.inf, math.inf,stop);        
        itera+=1      
        
        actualState = Linja(matrizoptima) 
        
    return actualState

## Método para jugar al juego (árbitro)


In [4]:
# Método que se encarga de gestionar los turnos y el correcto transcurso de la partida.
def play(state:Linja):
    gameOver = False
    html = state.get_html(False)
    display(HTML(html))
    while not state.gameOver():
        if state.turnoActual == 1 and not gameOver:
            entrada = input("Escriba dos números enteros separados por coma y espacio (<fila>, <columna>): ")

            # Dividir la entrada en dos números separados
            numeros = entrada.split(', ')

            # Verificar si se ingresaron exactamente dos números
            if len(numeros) == 2:
                try:
                    row = int(numeros[0])
                    col = int(numeros[1])

                    # Verificar si los números están dentro del rango especificado
                    if 0 <= row <= 7 and 0 <= col <= 5:
                        # Verificar si el movimiento es legal utilizando state.validateMov(row, col, False)
                        if state.validateMov(row, col, False):
                            if state.moverFicha(row, col, True):
                                html = state.get_html(False)
                                display(HTML(html))
                                if state.gameOver():
                                    state.endGame()
                                    gameOver = True
                            else:
                                html = state.get_html(True)
                                display(HTML(html))
                        else:
                            print("Error: Movimiento ilegal. Inténtalo de nuevo.")
                            html = state.get_html(False)
                            display(HTML(html))
                    else:
                        print("Error: Los números deben estar entre 0 y 7 para la fila y entre 0 y 5 para la columna.")
                except ValueError:
                    print("Error: Ingrese números enteros válidos.")
            else:
                print("Error: Ingrese dos números enteros separados por coma y espacio.")
        else:
            if not gameOver:
                state = performActionMinMax(state, 2)
                html = state.get_html(False)
                display(HTML(html))
            else:
                state.endGame()
                break


matrix = [[1, 1, 1, 1, 1, 1],
         [1, 0, 0, 0, 0, 2],
         [1, 0, 0, 0, 0, 2],
         [1, 0, 0, 0, 0, 2],
         [1, 0, 0, 0, 0, 2],
         [1, 0, 0, 0, 0, 2],
         [1, 0, 0, 0, 0, 2],
         [2, 2, 2, 2, 2, 2]]

tablero1 = Linja(matrix)
play(tablero1)


HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

Escriba dos números enteros separados por coma y espacio (<fila>, <columna>): 1,1
Error: Ingrese dos números enteros separados por coma y espacio.
Escriba dos números enteros separados por coma y espacio (<fila>, <columna>): 2, 2
Error: Movimiento ilegal. Inténtalo de nuevo.


HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

Escriba dos números enteros separados por coma y espacio (<fila>, <columna>): 1, 1
Error: Movimiento ilegal. Inténtalo de nuevo.


HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

...

HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

Escriba dos números enteros separados por coma y espacio (<fila>, <columna>): 0, 1


HTML(value='<style> img.game {width: 47px !important; height: 37px !important;}</style><table><tr><td><img cla…

KeyboardInterrupt: Interrupted by user